In [4]:
# Call RunPod served endpoint 
INTERNAL_PORT = 30000
POD_ID = "312aoreubi3q58"
BASE_URL = f"https://{POD_ID}-{INTERNAL_PORT}.proxy.runpod.net"

import requests 

try:
    # Call the root endpoint "/"
    response = requests.get(BASE_URL)
    print("Root endpoint response:", response.json())
    
    # Call the health endpoint "/health"
    health_response = requests.get(f"{BASE_URL}/health")
    print("Health check response:", health_response.json())
except Exception as e:
    print(f"Error: {e}")

Root endpoint response: {'message': 'API is running'}
Health check response: {'status': 'healthy'}


In [ ]:
# Dependency 
# pip install flashinfer -i https://flashinfer.ai/whl/cu124/torch2.4
# pip install orjson
# pip install sglang

from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

MODEL_PATH = "unsloth/Llama-3.2-11B-Vision-Instruct"
INTERNAL_PORT = 30000
CHAT_TEMPLATE = "llama_3_vision"


embedding_process = execute_shell_command(
    f"""
python3 -m sglang.launch_server --model-path {MODEL_PATH} \
    --port={INTERNAL_PORT} --chat-template={CHAT_TEMPLATE}
"""
)

wait_for_server(f"http://localhost:{INTERNAL_PORT}")


# Then we can access endpoint through OpenAI api
# Here the complication is we are using RunPod to connect things, BASE_URL is verylikely different from the default one 

MODEL_PATH = "unsloth/Llama-3.2-11B-Vision-Instruct"
INTERNAL_PORT = 30000
POD_ID = "312aoreubi3q58"
BASE_URL = f"https://{POD_ID}-{INTERNAL_PORT}.proxy.runpod.net"

# My assumetion is base_url = f"{BASE_URL}/v1" ...

from openai import OpenAI

client = OpenAI(base_url=f"{BASE_URL}v1", api_key="None")

response = client.chat.completions.create(
    model="unsloth/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print_highlight(response.choices[0].message.content)